In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data", one_hot=True)

In [ ]:
def weight(shape):
    ini = tf.truncated_normal(shape, stddev=0.1) #Outputs random values for weights on a truncated normal distribution with sd 0.1
    return tf.Variable(ini)

def bias(shape):
    ini = tf.constant(0.1, shape = shape) #Setting bias to 0.1
    return tf.Variable(ini)

def conv_layer(inpu, shape, strides=[1,1,1,1]): #Convolutional layer with stride of 1
    W = weight(shape)
    b = bias([shape[3]])
    return tf.nn.relu(tf.nn.conv2d(inpu, W, strides=strides, padding="SAME")+b) #Returning RELU on convolutional layer

def fully_connected(inpu, outpu): #Fully connected layer
    W = weight([int(inpu.get_shape()[1]), outpu])
    b = bias([outpu])
    return tf.matmul(inpu, W) + b

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, 784]) #Holds the images
x_image = tf.reshape(X, [-1,28,28,1]) #Reshape any image to 28x28x1 (the -1 will adjust to the size of the batch)
y = tf.placeholder(tf.float32, shape=[None,10]) #Holds the labels
keep_prob = tf.placeholder(tf.float32)

In [ ]:
conv1 = conv_layer(x_image, shape=[3,3,1,64]) #First convolutional layer
conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME") #First pooling layer
conv1_drop = tf.nn.dropout(conv1_pool, keep_prob = keep_prob)

conv2 = conv_layer(conv1_drop, shape=[5,5,64,128]) #Second convolutional layer
conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME") #Second pooling layer
conv2_drop = tf.nn.dropout(conv2_pool, keep_prob = keep_prob)

conv3 = conv_layer(conv2_drop, shape=[3,3,128,128]) #New third convolutional layer
conv3_pool = tf.nn.max_pool(conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
conv3_drop = tf.nn.dropout(conv3_pool, keep_prob = keep_prob)

conv3_flat = tf.reshape(conv3_drop, [-1, 4*4*128]) #Flattening the post-dropout Tensor to 50 x number of features
full1 = tf.nn.relu(fully_connected(conv3_flat, 1024))

full1_drop = tf.nn.dropout(full1, keep_prob=keep_prob) #Dropout layer for fully connected neurons

y_conv = fully_connected(full1_drop, 10)

In [ ]:
xentropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y))

train = tf.train.AdamOptimizer(1e-4).minimize(xentropy)

In [ ]:
correct = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
steps = 50000
batch_size = 50
final_accuracy = 0
saver = tf.trainSaver()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(steps):
        batch = mnist.train.next_batch(batch_size)
        sess.run(train, feed_dict={X: batch[0], y: batch[1], keep_prob: 0.5})
        
        if (i+1) % 5000 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={X: batch[0], y: batch[1], keep_prob: 1.0})
            test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
            print("Step {}, \tTraining accuracy {:.4f}, \tTest accuarcy {:.4f}".format((i+1), train_accuracy, test_accuracy))
            if test_accuracy > final_accuracy:
                final_accuracy = test_accuracy
                save_model = saver.save(sess, "models/CNN_MNIST_final.ckpt")
    print("\nFinal Accuracy {}".format(final_accuracy))

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random

with tf.Session() as sess:
    counter = 0
    saver.restore(sess, "models/CNN_MNIST_final.ckpt")
    for i in range(10000):
        digit = mnist.test.images[i].reshape(1,784)
        pred = tf.argmax(logits, 1)
        prediction = pred.eval(feed_dict = {X: digit})
        if prediction[0] != mnist.test.labels[i]:
            counter += 1
            print("Mistake " + str(counter))
            digit_image = digit.reshape(28,28)
            plt.imshow(digit_image, cmap = matplotlib.cm.binary, interpolation = "nearest")
            plt.axis("off")
            plt.show()
            print("Actual Label: " + str(mnist.test.labels[i]))
            print("Prediction: " + str(prediction[0]))
            print("\n")